In [1]:
import openai
import huggingface_hub as hf
import gptcache

from gptcache import cache
from gptcache.adapter import openai
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation

import time
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
cache.init()
cache.set_openai_key()


In [3]:
import time
def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']


def openai_request(question):
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages = [
            {
                'role': 'user',
                'content': question
            }
        ],
    )
    return response, time.time() - start_time

### OpenAI API original usage

In [4]:
question = 'What is github?'

answer, duration = openai_request(question)

print(f"Question: {question}")
print("Time consuming: {:2f}s".format(duration))
print(f"Answer: {response_text(answer)}")

Question: What is github?
Time consuming: 8.398907s
Answer: GitHub is a web-based platform that serves as a code hosting and version control system. It allows developers to collaborate on projects by providing a centralized repository where code can be stored, managed, and shared among team members. GitHub also provides features like issue tracking, wikis, and project management tools, making it widely popular among software developers and open-source communities.


### OpenAI API + GPTCache, exact match cache


In [5]:
question = 'What is github?'

for _ in range(2):
    answer, duration = openai_request(question)

    print(f"Question: {question}")
    print("Time consuming: {:2f}s".format(duration))
    print(f"Answer: {response_text(answer)}\n")

Question: What is github?
Time consuming: 7.013594s
Answer: GitHub is a web-based platform that serves as a code hosting and version control system. It allows developers to collaborate on projects by providing a centralized repository where code can be stored, managed, and shared among team members. GitHub also provides features like issue tracking, wikis, and project management tools, making it widely popular among software developers and open-source communities.

Question: What is github?
Time consuming: 0.001194s
Answer: GitHub is a web-based platform that serves as a code hosting and version control system. It allows developers to collaborate on projects by providing a centralized repository where code can be stored, managed, and shared among team members. GitHub also provides features like issue tracking, wikis, and project management tools, making it widely popular among software developers and open-source communities.



### OpenAI API + GPTCache, similar search cache

In [6]:
onnx = Onnx()
data_manager = get_data_manager(CacheBase('sqlite'), VectorBase("faiss", dimension=onnx.dimension))
cache.init(
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
)
cache.set_openai_key()

In [7]:
questions = [
    "what's github",
    "can you explain what GitHub is",
    "can you tell me more about GitHub",
    "what is the purpose of GitHub"
]

for question in questions:
    answer, duration = openai_request(question)

    print(f"Question: {question}")
    print("Time consuming: {:2f}s".format(duration))
    print(f"Answer: {response_text(answer)}\n")

Question: what's github
Time consuming: 11.965245s
Answer: GitHub is a web-based platform used to store and manage code repositories. It provides version control functionality and collaboration features that allow multiple developers to work on a project simultaneously. GitHub is widely used by developers and organizations to host open-source projects, collaborate on software development, and manage their code repositories. It offers various features such as code review tools, issue tracking, project management, and continuous integration and deployment.

Question: can you explain what GitHub is
Time consuming: 5.202634s
Answer: GitHub is a web-based platform used to store and manage code repositories. It provides version control functionality and collaboration features that allow multiple developers to work on a project simultaneously. GitHub is widely used by developers and organizations to host open-source projects, collaborate on software development, and manage their code reposito